## [실습 환경 Setting]
### 1. RealESRGAN 설치
- Download RealESRGAN pjt :
    ```
    pip install basicsr
    pip install facexlib
    pip install gfpgan
    git clone https://github.com/xinntao/Real-ESRGAN.git
    <python_home>/Lib/site-packages 위치에 Real-ESRGAN directory 통 copy
    cd <python_home>/Lib/site-packages/Real-ESRGAN
    pip install -r requirements.txt
    python setup.py develop
    ```
### 2. RealESRGAN 용 AI Model 설치
- Download RealESRGAN model(Restoring model):
    https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth
    
    저장 위치 : python code 에 model 위치 명시

In [4]:
import torch
import cv2
import numpy as np
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

def upscale_image(input_path, output_path, model_path, scale=2):
    # 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 모델 생성
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=scale)
    
    # 업스케일러 생성 (half=False로 설정하여 FP32 사용)
    upscaler = RealESRGANer(scale=scale, model_path=model_path, model=model, tile=400, tile_pad=10, pre_pad=0, half=False)
 
    # 이미지 로드 (OpenCV 사용)
    img = cv2.imread(input_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 변환
    
    # 업스케일링 수행
    output, _ = upscaler.enhance(img, outscale=scale)
    
    # 결과 저장 (OpenCV 사용)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)  # RGB에서 BGR로 변환
    cv2.imwrite(output_path, output)
    
    print(f"업스케일링된 이미지가 {output_path}에 저장되었습니다.")

if __name__ == "__main__":
    input_path = r'C:\pypjt\images\hugging.jpg'  # 입력 이미지 경로
    output_path = "upscaled_image.png"  # 출력 이미지 경로
    model_path = r"C:\pypjt\restore\Lib\site-packages\Real-ESRGAN\weights\RealESRGAN_x4plus.pth"  # Real-ESRGAN 모델 파일 경로
    
    upscale_image(input_path, output_path, model_path, scale=4)

	Tile 1/2
	Tile 2/2
업스케일링된 이미지가 upscaled_image.png에 저장되었습니다.


In [ ]:
import cv2
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from insightface.utils import face_align

# RealESRGAN을 이용한 업스케일링
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
upsampler = RealESRGANer(
    scale=2, model_path='weights/RealESRGAN_x4plus.pth', model=model, tile=400, tile_pad=10, pre_pad=0, half=True
)

# 이미지 불러오기
image = cv2.imread(r'C:\pypjt\images\hugging.jpg')

# 리사이즈된 이미지를 업스케일링
upscaled_image, _ = upsampler.enhance(image, outscale=2)

# 결과 저장
cv2.imwrite('upscaled_image.jpg', upscaled_image)


In [ ]:
import torch
import cv2
import numpy as np
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.realesrgan_utils import RealESRGANer
from typing import Literal

UPSCALE_MODEL_X2 = r"C:\pypjt\restore\Lib\site-packages\Real-ESRGAN\weights\RealESRGAN_x2plus.pth"
UPSCALE_MODEL_X4 = r"C:\pypjt\restore\Lib\site-packages\Real-ESRGAN\weights\RealESRGAN_x4plus.pth"

def upscale_image(input_image: np.ndarray, scale: Literal[2, 4] = 2) -> np.ndarray:
    """
    이미지를 업스케일링하는 함수.
    
    :param input_image: ndarray 타입의 입력 이미지
    :param model_path: 모델 파일 경로
    :param scale: 업스케일 배율 (기본값: 2)
    :return: 업스케일링된 ndarray 타입의 이미지
    """
    # 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    # Upscaing Model 선택
    model_path = UPSCALE_MODEL_X2 if scale == 2 else UPSCALE_MODEL_X4

    # 모델 생성
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=scale)
    
    # 업스케일러 생성 (half=False로 설정하여 FP32 사용)
    upscaler = RealESRGANer(scale=scale, model_path=model_path, model=model, tile=400, tile_pad=10, pre_pad=0, half=False)
    
    # 입력 이미지가 BGR 형식일 경우, RGB로 변환
    if input_image.shape[-1] == 3:  # 이미지가 컬러일 경우
        input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    
    # 업스케일링 수행
    output, _ = upscaler.enhance(input_image, outscale=scale)
    
    # 결과를 BGR로 다시 변환하여 반환
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    
    return output

In [ ]:
import cv2
import numpy as np

# 입력 이미지 로드
input_img = cv2.imread("city_image.jpg")

# upscaling 함수 호출
upscaled_img = upscale_image(input_img, scale=4)

# 결과 저장
cv2.imwrite('upscaled_city.jpg', upscaled_img)